In [1]:
import os

from langchain_openai import OpenAI, OpenAIEmbeddings
from pinecone import Pinecone

In [2]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot'

In [5]:
with open('.env') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

In [6]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX')

In [7]:
pincone = Pinecone(api_key=PINECONE_API_KEY)
index = pincone.Index(PINECONE_INDEX)

In [8]:
from langchain_pinecone import Pinecone

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536)
text_field = "text"

query = "What are the specific features or aspects that users appreciate the most in our application?"

vectorstore = Pinecone(
    index, embeddings, text_field, namespace='ns-0'
)

In [10]:
vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='google_store_review_comments: Awesome UI, best music app out there!', metadata={'author_app_version': '1.1.0.91', 'author_name': 'A Google user', 'pseudo_author_id': '286593453219054880269', 'review_id': '672a155a-e81e-4d28-bdeb-a74c031bc072', 'review_likes': '1', 'review_rating': '5', 'review_timestamp': '2014-05-27 15:17:20', 'row': 3.0, 'source': '672a155a-e81e-4d28-bdeb-a74c031bc072'}),
 Document(page_content='google_store_review_comments: I enjoy the awesome UI of this app, and it has all the music one can ask for.', metadata={'author_app_version': '1.1.0.91', 'author_name': 'A Google user', 'pseudo_author_id': '234382942865437071667', 'review_id': 'bfa8876b-470e-4640-83a7-77427f7f37e8', 'review_likes': '4', 'review_rating': '5', 'review_timestamp': '2014-05-27 14:36:02', 'row': 1.0, 'source': 'bfa8876b-470e-4640-83a7-77427f7f37e8'}),
 Document(page_content='google_store_review_comments: Excellent music app', metadata={'author_app_version': '1.1.0.112', 'au

In [11]:
from langchain.chains import RetrievalQA

In [12]:
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct', openai_api_key=OPENAI_API_KEY)

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': 'What are the specific features or aspects that users appreciate the most in our application?',
 'result': ' The users appreciate the awesome UI and the variety of music available in the app. They also enjoy the overall experience and the continuous improvement of the app.'}